In [1]:
# GROUP 2 CODE
# Members: Amiri, Ebru, Madison, Cailyn
# Github: https://github.com/AmiriHayes/rockphysicsproject

"""
Goals: 
- Develop simulated 3D data to work with [✓]
- Simplify dataset (into graph network) [✓]
- Predict fluid flow on simulated data [✓]

To install, run "pip install -r requirements.txt" in terminal 
Then ensure you use "pip install git+https://github.com/PMEAL/porespy.git@dev" to install latest porespy!

Code References:
1. polydisperse_spheres: https://porespy.org/examples/generators/reference/polydisperse_spheres.html
2. open_pnm_to_im: https://porespy.org/modules/generated/generated/porespy.io.openpnm_to_im.html
3. predict fluid flow: https://openpnm.org/examples/applications/network_extraction.html

"""

'\nReferences:\n\n'

In [ ]:
# imports:

import numpy as np
import porespy
import openpnm as op
import matplotlib.pyplot as plt
from tqdm import tqdm
ps.visualization.set_mpl_style()
np.random.seed(10)

In [9]:
# pore-reduction:

im = ps.generators.blobs(shape=[400, 400], porosity=0.6, blobiness=2)
fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(im)

snow_output = ps.networks.snow2(im, voxel_size=1)

In [ ]:
# convert to openim graph network:

pn = op.io.network_from_porespy(snow_output.network)

fig, ax = plt.subplots(figsize=[5, 5])
ax.imshow(im.T, cmap=plt.cm.bone)
op.visualization.plot_coordinates(ax=fig,
                                  network=pn,
                                  size_by=pn["pore.inscribed_diameter"],
                                  color_by=pn["pore.inscribed_diameter"],
                                  markersize=200)
op.visualization.plot_connections(network=pn, ax=fig)
ax.axis("off")

In [ ]:
data = {
    'shape': {
        'x': im.shape[0],
        'y': im.shape[1],
        # 'z': im.shape[2],
    },
    'resolution': 5.345e-6,
    'porosity': 19.6,
    'permeability': {
        'Kx': 1360,
        'Ky': 1304,
        'Kz': 1193,
        'Kave': 1286,
    },
    'formation factor': {
        'Fx': 23.12,
        'Fy': 23.99,
        'Fz': 25.22,
        'Fave': 24.08,
    },
}

In [ ]:
# define network statististics

pn['pore.diameter'] = pn['pore.equivalent_diameter']
pn['throat.diameter'] = pn['throat.inscribed_diameter']
pn['throat.spacing'] = pn['throat.total_length']

pn.add_model(propname='throat.hydraulic_size_factors',
             model=op.models.geometry.hydraulic_size_factors.pyramids_and_cuboids)
pn.add_model(propname='throat.diffusive_size_factors',
             model=op.models.geometry.diffusive_size_factors.pyramids_and_cuboids)
pn.regenerate_models()

In [ ]:
# check network is valid

h = op.utils.check_network_health(pn)
print(h)

op.topotools.trim(network=pn, pores=h['disconnected_pores'])
h = op.utils.check_network_health(pn)
print(h)

In [ ]:
# define necessary gas / liquid properties and physics models 

gas = op.phase.Phase(network=pn)
gas['pore.diffusivity'] = 1.0
gas['pore.viscosity'] = 1.0
gas['throat.entry_pressure'] = 0
gas['throat.surface_tension'] = 1.0
gas['throat.contact_angle'] = 0

gas.add_model(propname="pore.viscosity", model=op.models.misc.constant, value=1.0)
gas.add_model(propname="pore.diffusivity", model=op.models.misc.constant, value=1.0)
gas.add_model(
    propname="throat.diffusive_conductance",
    model=op.models.misc.constant, value=1.0
)

In [ ]:
# calculate the flow through simulated dataset

fd = op.algorithms.FickianDiffusion(network=pn, phase=gas)
fd.set_value_BC(pores=pn.pores('xmin'), values=1.0)
fd.set_value_BC(pores=pn.pores('xmax'), values=0.0)
fd.run()

dC = 1.0
L = (data['shape']['x'] + 6) * data['resolution']
A = data['shape']['y'] * data['shape']['z'] * data['resolution']**2
Deff = fd.rate(pores=pn.pores('xmin')) * (L / A) / dC
F = 1 / Deff

print(f"The Formation factor of the extracted network is {F[0]}")
print(f"This compares to a value of {data['formation factor']['Fx']} from DNS")
np.testing.assert_allclose(F, data['formation factor']['Fx'], rtol=0.09)